## Deezer playlist dataset et recomandandation de morceaux avec word2vec

Dans ce mini projet nous allons mettre au point un réseau word2vec et nous en servir pour construire un outils de complétion de playlist (suggestion de morceaux). Les données sont hébergée sur le dépot suivant : http://github.com/comeetie/deezerplay.git. Pour en savoir plus sur word2vec et les données que nous allons utiliser vous pouvez lire les deux références sivantes :

- Efficient estimation of word representations in vector space, Tomas Mikolov, Kai Chen, Greg Corrado, and Jeffrey Dean. (https://arxiv.org/abs/1301.3781)
- Word2vec applied to Recommendation: Hyperparameters Matter, H. Caselles-Dupré, F. Lesaint and J. Royo-Letelier. (https://arxiv.org/pdf/1804.04212.pdf)

Les éléments que vous devez réaliser sont mis en évidence en <span style="color:red">rouge</span> 


### Préparation des données
Les données sont sous la forme d'une liste de playlist. Chaque playlist est elle me une liste avec l'identifiant deezer du morçeau suivi de l'identifiant de l'artiste.

In [1]:
# chargement des données de playlist
import numpy as np
data = np.load("./music_2.npy",allow_pickle=True)
[len(data), np.mean([len(p) for p in data])]

[100000, 24.21338]

Le jeu de données sur lequel nous allons trvailler contient 100000 playlist qui sont composeer d'en moyenne 24.1 morceaux. Nous allons commencer par ne conserver que les identifiants de morceau. 

In [2]:
# separation des ids de morçeau et d'artist
playlist_track = [list(filter(lambda w: w.split("_")[0]==u"track",playlist)) for playlist in data]
playlist_artist = [list(filter(lambda w: w.split("_")[0]==u"artist",playlist)) for playlist in data]

In [3]:
# nombre de morceaux != ?
tracks = np.unique(np.concatenate(playlist_track))
Vt = len(tracks)
Vt

338509

In [4]:
playlist_track[1]

['track_354917971',
 'track_354917981',
 'track_354917991',
 'track_354918001',
 'track_354918011',
 'track_405992352',
 'track_383552261',
 'track_397746652',
 'track_385499681',
 'track_402932992',
 'track_398786652',
 'track_357037431',
 'track_378113751',
 'track_354918011',
 'track_403520542',
 'track_393157072',
 'track_354165291',
 'track_357083271',
 'track_361171651',
 'track_145501426',
 'track_397141342',
 'track_145370202']

Le nombre de morceaux différents dans ce data-set est assez élevé avec plus de 300 000 morceaux.

### Création d'un dictionnnaire de morceau
Nous allons affecter a chaque morceau un entier qui nous servira d'identifiant unique et d'entrée pour notre réseau. Pour économiser un peu nos ressources nous allons travailler dans ce TP que sur les morceaux qui apparaissent dans au moins deux playlists.

In [5]:
# nombre d'occurence de chaque morceaux ?
track_counts = dict((tracks[i],0) for i in range(0, Vt))
for p in playlist_track:
    for a in p:
        track_counts[a]=track_counts[a]+1;

In [6]:
# Filtrage des morceaux peu fréquent pour gangner un peu de temps au vue de nos ressource en temps de calcul  
playlist_track_filter = [list(filter(lambda a : track_counts[a]> 1, playlist)) for playlist in playlist_track]
# recupération des comptage
counts  =  np.array(list(track_counts.values()))
# trie
order = np.argsort(-counts)
# création de notre liste d'identifiant deezer
tracks_list_ordered = np.array(list(track_counts.keys()))[order]
# Taille de notre vocabulaire = nombre de morçeau conservés
Vt=np.where(counts[order]==1)[0][0]
# construction d'un dict id_morceaux id [0,Vt]
track_dict = dict((tracks_list_ordered[i],i) for i in range(0, Vt))
# conversion des playlist en liste d'entier
corpus_num_track = [[track_dict[track] for track in play ] for play in playlist_track_filter]

In [7]:
corpus_num_track[1]

[313,
 436,
 259,
 521,
 66,
 235,
 152,
 56,
 116,
 71,
 505,
 113,
 217,
 66,
 277,
 527,
 348,
 848,
 378,
 465,
 532,
 2016]

### Création des ensembles d'apprentissage de test et de validation

Pour apprendre les paramètre de notre méthode nous allons conserver les $l-1$ premiers morceaux de chaque playlist (avec $l$ la longueur de la playlist) pour l'apprentissage. Pour évaluer les performances de completion de notre méthode nous conservons pour chaque playlist les deux derniers morceaux. L'objectif sera de trouver le dernier a partir de l'avant dernier. 



In [8]:
# ensemble de test et d'apprentissage
index_tst = np.random.choice(100000,20000)
index_val = np.setdiff1d(range(100000),index_tst)
# le debut de chaque playlist est conservé pour l'apprentissage
play_app  = [corpus_num_track[i][:(len(corpus_num_track[i])-1)] 
             for i in range(len(corpus_num_track)) if len(corpus_num_track[i])>1]
# les deux derniers élemnts pour le test et la validation
play_tst  = np.array([corpus_num_track[i][(len(corpus_num_track[i])-2):len(corpus_num_track[i])] 
             for i in index_tst if len(corpus_num_track[i])>3])
play_val  = np.array([corpus_num_track[i][(len(corpus_num_track[i])-2):len(corpus_num_track[i])] 
             for i in index_val if len(corpus_num_track[i])>3])[:10000]


In [9]:
# import de Keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense,Flatten
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.preprocessing.sequence import skipgrams

2022-04-18 17:43:58.024648: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-18 17:43:58.024664: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### hyper-paramètres de word2vec :

La méthode word2vec fait intervennir un certains nombre d'hyper paramètres. Nous allons les définirs et leurs donner des première valeurs que nous affinerons par la suite:

In [10]:
# dimension de l'espace latent
vector_dim = 30
# taille de la fenêtre de voisinage
window_width = 3
# sur-échantillonage des exemples négatifs
neg_sample = 5
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.5
# coeff pour le subsampling
sub_samp = 0.00001

### Création des tables de probabilité de tirage (lissée) et non lissée

Pour tirer les exmples négatif nous avons besoin des fréquence lissé de chaque morceau dans notre dataset. De même pour sous échantilloner les morceaux très fréquents nous avons besoin des fréquence brutes. Nous allons calculer ces deux vecteurs.

In [11]:
# recupération des comptage
counts = np.array(list(track_counts.values()),dtype='float')[order[:Vt]]
# normalisation
st =  counts/np.sum(counts)
# lissage
st_smooth = np.power(st,samp_coef)
st_smooth = st_smooth/np.sum(st_smooth)

### Construction du réseau word2vec

Un réseau word2vec prend en entrée deux entiers correspondant à deux morceaux, ceux-ci sont plonger dans un espace latent de dimension (vector_dim) grâce a une couche de type embedding (vous devrez utilisez la même couche pour projeter les deux morceaux). Une fois ces deux vecteurs extraits le réseau doit calculer leur produit scalaire normaliser appleler cosine distance : 

$$cos(\theta_{ij})=\frac{z_i.z_j}{||z_i||||z_j||}$$ 

Pour réaliser ce traitement vous utiliserez une couche "Dot" pour "dot product". Le modèle utilise ensuite une couche de type sigmoid pour produire la sortie. Cette sortie vaudra 0 lorsque les deux morceaux sont des morceaux tirés aléatoirement dans l'ensemble du jeu de donnée et 1 lorsqu'il aurront était extraits de la même playslist. <span style="color:red">A vous de créer le modèle keras Track2Vec correspondant à cette architecture.</span>

In [12]:
# entrée deux entier (couple de morceaux)
input_target = Input((1,), dtype='int32')
input_context = Input((1,), dtype='int32')

# a vous de compléter
embedded = Embedding(Vt, vector_dim, name="embedding-input")
embedded_target = embedded(input_target)
embedded_context = embedded(input_context)
dot_product = Dot(axes=1, normalize=True, name="dot")([embedded_target, embedded_context])
output = Dense(1, activation='sigmoid',name="classif")(dot_product)

# definition du modèle
Track2Vec = Model(inputs=[input_target, input_context], outputs=output)
Track2Vec.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy"])

2022-04-18 17:44:01.194927: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-18 17:44:01.194975: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-18 17:44:01.194998: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tom-GF63-Thin-10SC): /proc/driver/nvidia/version does not exist
2022-04-18 17:44:01.195868: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-18 17:44:01.202739: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14788920

In [13]:
Track2Vec.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding-input (Embedding)    (None, 1, 30)        3697230     ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 dot (Dot)                      (None, 30, 30)       0           ['embedding-input[0][0]',    

### Création du générateur de données

Pour apprendre la couche de projection au coeur de notre modèle nous allons construire une générateur d'exemples positifs et négatifs de pair de morceaux proche ou aléatoires issues de nos données d'entrainement. La fonction suivante va permettre de générer de tels exemples a partir d'une playlist (seq) fournies en entrées. Cette fonction va tout d'abord construire tout les couples de morceau pouvant être extraient de la séquences s'ils se situent à moins de (windows) disance l'un de l'autres. Ces paires constitueront les paires positives. Les paires concernant deux morceaux très fréquents seront supprimer avec une probabilité qui dépendra de leur fréquences. Enfin un nombre d'exemple négatifs (correpondant neg_samples * nombre d'exemple positif) vont être tirés aléatoirement en utilisant la table de tirage (neg_sampling_table). 

In [14]:
# fonction générant les données associé a une séquence
# seq : séquence d'entrée
# neg_samples : nombre d'exemple négatif générés par example positif
# neg_sampling_table : probabilité de tirage des exemples négatif
# sub sampling_table : probabilité servant a sous échantilloner
# sub_t : paramètre de sous échantillonage
def word2vecSampling(seq,window,neg_samples,neg_sampling_table,sub_sampling_table,sub_t):
    # taille du vocabulaire
    V = len(neg_sampling_table)
    # créations des paires positives a partir de la séquence
    positives = skipgrams(sequence=seq, vocabulary_size=V, window_size=window,negative_samples=0)
    ppairs    = np.array(positives[0])
    # sous échantillonage
    if (ppairs.shape[0]>0):
        f = sub_sampling_table[ppairs[:,0]]
        subprob = ((f-sub_t)/f)-np.sqrt(sub_t/f)
        tokeep = (subprob<np.random.uniform(size=subprob.shape[0])) | (subprob<0)
        ppairs = ppairs[tokeep,:]
    nbneg     = ppairs.shape[0]*neg_samples
    # tirage des paires négatives
    if (nbneg > 0):
        negex     = np.random.choice(V, nbneg, p=neg_sampling_table)
        negexcontext = np.repeat(ppairs[:,0],neg_samples)
        npairs    = np.transpose(np.stack([negexcontext,negex]))
        pairs     = np.concatenate([ppairs,npairs],axis=0)
        labels    = np.concatenate([np.repeat(1,ppairs.shape[0]),np.repeat(0,nbneg)])
        perm      = np.random.permutation(len(labels))
        res = [pairs[perm,:],labels[perm]]
    else:
        res=[[],[]]
    return res

<span style="color:red">Utilisez cette fonction pour constuire un générateur "track_ns_generator" de données qui va générer des exemples positifs et négatifs à partir de "nbm" playlists tirées aléatoirement dans le jeu de données "corpus_num" fournis en entrée. </span>

In [15]:
# définition du générateur de couple de morceaux (y=0 <-> aléatoire, y=1 <-> proche dans une playlist)
import random
def track_ns_generator(corpus_num,nbm):
    #while True :
    # tirage de nbm playlist dans corpus_num
    x1 = []
    x2 = []
    y = []
    for i in range(nbm) :
        playlist = random.choice(corpus_num)
        # création des données x et y 
        xa, ya = word2vecSampling(playlist, window_width, neg_sample, st, st_smooth, sub_samp)
        for elem in xa :
            x1.append(elem[0])
            x2.append(elem[1])
        for elem in ya :
            y.append(elem)
    x = [np.array(x1), np.array(x2)]
    y = np.asarray(y).astype('float32').reshape((-1,1))
    print(len(y))
    print(len(x1))
    return (x,y) 

track_ns_generator(play_app, min_batch_size)

13362
13362


([array([ 9590,  9590,   486, ..., 18830, 47629, 53499]),
  array([33875, 10400,  6604, ..., 53499,   317, 39365])],
 array([[0.],
        [1.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]], dtype=float32))

## Apprentissage 
Vous devriez maintenant être en mesure d'apprendre votre premier modèle avec le code suivant. Cela devrait durer entre 15 et 30 min.

In [16]:
import tensorflow
xx, yy = track_ns_generator(play_app,min_batch_size)
tensorflow.config.run_functions_eagerly(True)
hist = Track2Vec.fit(xx, yy, steps_per_epoch = 200,epochs=60)

13170
13170
Epoch 1/60
  1/200 [..............................] - ETA: 21s - loss: 0.8510 - accuracy: 0.5061

/home/tom/anaconda3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
2022-04-18 17:44:14.263201: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14788920 exceeds 10% of free system memory.
2022-04-18 17:44:14.265684: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14788920 exceeds 10% of free system memory.


200/200 [==============================] - 11s 54ms/step - loss: 0.7807 - accuracy: 0.5275
Epoch 2/60
200/200 [==============================] - 13s 67ms/step - loss: 0.6363 - accuracy: 0.6571
Epoch 3/60
200/200 [==============================] - 11s 57ms/step - loss: 0.5722 - accuracy: 0.8191
Epoch 4/60
200/200 [==============================] - 12s 58ms/step - loss: 0.5369 - accuracy: 0.8333
Epoch 5/60
200/200 [==============================] - 13s 66ms/step - loss: 0.5125 - accuracy: 0.8333
Epoch 6/60
200/200 [==============================] - 15s 75ms/step - loss: 0.4945 - accuracy: 0.8333
Epoch 7/60
200/200 [==============================] - 12s 60ms/step - loss: 0.4812 - accuracy: 0.8333
Epoch 8/60
200/200 [==============================] - 12s 58ms/step - loss: 0.4715 - accuracy: 0.8333
Epoch 9/60
200/200 [==============================] - 12s 59ms/step - loss: 0.4646 - accuracy: 0.8333
Epoch 10/60
200/200 [==============================] - 13s 63ms/step - loss: 0.4598 - accurac

## Sauvegarde de l'espace latent
Nous pouvons une fois l'apprentissage effectué sauvegarder la position des morceaux dans l'espace latent avec le code suivant:

In [17]:
# récupérations des positions des morceaux dans l'espace de projection
vectors_tracks = Track2Vec.get_weights()[0]
with open('latent_positions.npy', 'wb') as f:
    np.save(f, vectors_tracks)

Et nous pouvons la recharger avec le code suivant :

In [18]:
vectors_tracks=np.load("latent_positions.npy")

## Utilisation en complétion et évaluation
Nous pouvons maintenant nous servir de cet espace pour faire des suggestions. <span style="color:red">Construisez une fonction predict_batch qui prend en entrée un vecteur de numéro de morceaux (seeds), (s) un nombre de proposition a faire, les vecteurs des morceaux dans l'espace latent X et un kd-tree permettant d'accélérer les calculs de plus proche voisins. Pour faire ses propisitions cette fonctions retournera les indices des s plus proche voisins de chaque seeds. </span> Pour que ces predictions ne prennent pas trop de temps vous vous servirez d'un kd-tree (disponnible dans scikit learn) pour accélrer la recherche des plus proche voisins.

In [19]:
from sklearn.neighbors import KDTree
kdt = KDTree(vectors_tracks, leaf_size=10, metric='euclidean')

In [20]:
def predict_batch(seeds,k,X,kdt):
    #return [kdt.query(X[i:i+1], k = 10) for i in seeds]
    preds = []
    for i in seeds : 
        x = kdt.query(X[i:i+1], k = 10)[1]
        preds.append(x)
        print(x)
    return preds

<span style="color:red">Utilisez cette fonction pour proposer des morceaux pour compléter les playlist du jeu de données de validation (les seeds correspondent à la première colone de play_val).</span>

In [21]:
indexes = predict_batch(play_val[:,0],10,vectors_tracks,kdt)

[[ 79029  23883  49363  18165  30146  66260  24528  30389  44521 117889]]
[[ 79202  36271   2634  48793  23956  75894  92273  99124 114117   1281]]
[[11720 63081 96213 42185  5211  4341 55356 32868 20832  3533]]
[[ 21803  47263   2683 118853  29958  10812  68663 121126 109671 122299]]
[[ 68426 114529  93158  35474  38384  16943 117469   1715  86837  41811]]
[[   569  87247  49770  68888  76843  90879 106049  32625 105584  33343]]
[[ 45641  24640 121357  31728  81132  43419  69840   2128  98473  84768]]
[[59046 83660  5019 13574 46805 84884 68062  1301 40463 50730]]
[[   152 104852   8333  64409 120533 105910  52615  77937  46547 116601]]
[[ 11736   8869  84847  78670  71298 104457 101787  83863  30296  67493]]
[[    30  36447 103078  26143  36270  50475  60941  53278  67808  20723]]
[[  3478 111686  48060  22877  48610 118386  14765  53857 110015  69551]]
[[ 3755 25334 48170 31276 41275  4109 13154 16735 83098 94278]]
[[  3120  22131   7869  17396 117462  12700  19951 114282 101392  24

[[   387  91915  24665  80957  68882  96404 120438  15055  52606  26280]]
[[114819  37623  72651 123058  52457  60630 102726  90144 103892 118747]]
[[ 50678   7444  18924  88747  12243 116688   1486  92182 108148  47872]]
[[  2848  65256 113856  11996  17176  14040  67341  59588  50302   3001]]
[[ 21442 110926 106068  99646  68991   9762  31730  81422  20098  99907]]
[[    27  34348  49585  69165  97151 107583  42807 102756  83625  77978]]
[[73117 11567 81135 29252  7405 48493 52032  1940 55928 81209]]
[[ 74921  17973 103831  92704  53676  54610  88457  93262  29926  28030]]
[[ 12258  82722  24595  74321  43975   5553 114340  90469  20328  36320]]
[[101288  64987   2721  71137  54139 115166 116453  66099  71677  23718]]
[[ 70392  31378  27231  59352  16769  96508 116341 117570  48993 121580]]
[[100798  30180  38870   9472  34843  12524  69738  33770  85404  73050]]
[[    51 116953 109175 116325  97122  71988 105649  57182  11172  57317]]
[[ 97675 104283  44433  81110  59853  79826  371

[[ 24548  26142 101987  50581   1943  58675  63309  26817  52813  24347]]
[[   92 39538 18863  8100 81416 69798 71546 23411 56055 12077]]
[[    28  43325 111185  18221  86678 116850  95074  60129  39513  12571]]
[[ 75745  70317  62026 117776  28357  26696 108523 112760  49134  70672]]
[[ 23589  54061  19572  84820  17279  29963 106841  64016  10783  57690]]
[[  3257  90476  59058 114362  90574  94725  88578  20788  63305  49820]]
[[  2101  17715  85399  36460 118283  39526  11323  89809  93583  93429]]
[[  5708  88739 114141  44395  20756 106906  90828  12434  80304  80826]]
[[ 25331  10114  56151  97209 116919   7109  46016  46226  51623  91543]]
[[  3947  86089  29873 105435 117396   4247  57396  24759  54709  60067]]
[[  1362  94196  31700 111627 120287  59650  83384  42564  50462  26015]]
[[115847  60186  51902  83386  87416   6696  24912 113628   8209  42219]]
[[ 31263 105697   7580 106880  48401  48054   6393  18856  15518  93709]]
[[ 20626  46853  38945 105859  99726  37894  252

[[  5060  96560   2273  91392  50966    211 105753  45162 119281  60488]]
[[   126 116346  51137  48446  52413  17669  28215  85746   2551  30151]]
[[ 10267  54598 112201  17783  94406 104867  73236 117836   7073  49281]]
[[  1448  75036 117089 121812  38228  14585  47300  58538  75120  59586]]
[[84121  7005 49597 20458 17779 20767 86678 12795 66793 13156]]
[[ 54118  45562 117410  95753  16711  95246  72865  42114 112409  77909]]
[[  2599  21816    624 121640   5755  32714  50741  96836  59342  56801]]
[[   360  88057  48391  15705  36223  56813  42805 114865  29770  90537]]
[[ 17069 100190  69186  10601  79589   3482   4375  75218  34506  15093]]
[[88995 67088 47209 36857  6894 79143  2491 88664 15987 53280]]
[[  1122  75031   2652  10993  31171  27662  32706  67775 110308  63154]]
[[  3182  55161 103365   4659  12909  53581  36561  75457  19796  24235]]
[[  8846 120893  37887  74481 117484 100561  59297 103983 119655 108851]]
[[ 28530 102303  76757  17475  56910  42989   1210  23963 

[[ 36525  25421  63001  83884  59990 103310  22789   5646  88001  90178]]
[[ 28601  67111 119646  20115 123199 106824  31121  20025  74765  27920]]
[[ 68715 111969 116722  62231  72639  33159  14323  89892  93698   1233]]
[[ 22261   1245 102546  70896  63106  75282  74939  32528  52997  70935]]
[[  2511  24055 109962  48575  10917  29942  75068   3640 119077  57981]]
[[  7088 106343  69180  76416  97334  41754  98595  89408 123127  44813]]
[[  9256 102351  49155   9303  62153   7654  37894 104194  24188    959]]
[[ 14773  42483  43328  65824 103514 103803  47602  35277  58415  86036]]
[[  2083  56342  20420  70121  40302  77223 115334 121513 108124  56883]]
[[   92 39538 18863  8100 81416 69798 71546 23411 56055 12077]]
[[  7415   3364 106053  89745 116364  99965  67371 110361  54620  42911]]
[[108381  48636  16590  40613  71097   2054  89111  96146  31949 118293]]
[[ 10525 106503  93096  94425  47131  64888   1117  26503 110891  93859]]
[[  5900  59328  84588  61346  58959  84331 1214

...

[[ 8381 23413  5989  9611  9938 18891 60088 53396 96664  5804]]
[[   993  10023 122765 120548  21422  81222  66360  39005  32427  82692]]
[[ 99090  71959  97216  42768  80860  75915  84784 117640  35631  16658]]
[[  7319 106369  14583  77809 122932  10215  27822 115402  14033 111666]]
[[ 22571  66492  49144  87142  17660  39130  70045 102604  19494  12612]]
[[  1839  87859  31775  84497  40430  57746 121454  74055  58434  91750]]
[[  1076   3950  14907  33667  96025  20812  50802 107078  17225  69924]]
[[ 64052      1 103280 103911 110519  81475  39258  76068  39004  67875]]
[[  2458 114323 111266 114436 120407 105435   8332  70343  62692  84867]]
[[   354   3196  79519  96407  14683  38570  60518 118085  25852  74410]]
[[ 22101  90471 107706  70816  76400  75373  72025  40628  18231 102062]]
[[115465  74327  68769   6667  20079  74648  60892 101638  83009  34227]]
[[ 35926  47030 109184   6764  94633   6939  87056  45075  24601  41230]]
[[ 69277 108006  73763  66229 110707  34962 1213

[[ 9966 16389 77243 60480 53902 55890  1031 25335 53220 80831]]
[[ 68204  92077   6414  90378  11486  65971 111289 117949  75295  62439]]
[[113215  12497  35285 113370  49067 113770  56732  16898  75372  98520]]
[[ 24042  99536  52989 104324  83688  31457  49955 121553 108491  24865]]
[[   290 107978  97752  70219 112802  88443 100235  87022  69031   5249]]
[[   131 112806 110336  24397  56318  39938  33693  78185 108433  58075]]
[[ 15150  57257  14037 117269  46893 121663  87982 112721  62455  89954]]
[[ 32694 100397 121385  69290  35542  89647  61374   9077   5288  29517]]
[[  1721  65460  45540  37581 118662    259  89788 122702  79046  98702]]
[[   104  52508  18813  60777 115330    830   3013   4126  54725  52271]]
[[ 12870   7934 123068  78162 110981  73591  37376  32358 112327 110679]]
[[  3116 105190  20339  21492 106357  51037  50117  39060  11370  81718]]
[[     4 117337  75477 119097  34565  38948  36904  71866  47961  53128]]
[[  1474  57628  91692  49902  37416  76111  357

[[  7465  37705  49654  41025  74432  72773   3554  63293 113412  10902]]
[[ 76975 106229  19491  98983  49042  38980  82754  85257  54513  29392]]
[[    29  66454  47460  27075  95691  82739 103676   9293  39431  94111]]
[[    13  32529  21645 100457   7845   3590  50292  10061    636  73952]]
[[ 72167  85007  20350  95875 101256  34181  88747   8520  50080  72802]]
[[46432 17972  1477 30654 52045 20395 48588 42858 53846 54186]]
[[   743  70717  48318  11075  74316  14696  24387 120550 118357  21078]]
[[  6467  40007  61451  31257  73486 100262  10481  94840  40453  58201]]
[[    39 112671  87224 104072  99507  98499  84101  50605  56425  75044]]
[[  1820 122194  29498 103910  49063  63700  58774  54964  42976 100218]]
[[ 22260  61967  71734  31161 113440  45643  11685  78888  52785  27197]]
[[ 6524 77991  2602 56489 34397 16140 99555 91015 93854 87367]]
[[ 33968 110013   4161  12820 112363  10655  69342  60948 104566  52038]]
[[20771 67361 95791 69907 62900 34685  8633 84075 96676 86

[[ 25449 119529 101445 111736  69318  54820  60096  69319 120731  55218]]
[[ 6298 30378 49137 52032 34377 25912 85592 71597 81135 71636]]
[[ 32787 115875  54829 112329   7391  75202 123098  12337  14967  98166]]
[[   470  27672  48193 120706  15723  17771  42685  37614  32699  30055]]
[[ 38379  75389 116057 101163  17805 121423  30454 101648  49173  23837]]
[[ 17291 119062  28964 105218  93505 112923  40922  85584  32923 122042]]
[[ 18728  84971  12529  45761  46286  37711  31540  55982  55567 114468]]
[[  1104 117101  50123 112329  42775 108499  78812 106409  62581  24596]]
[[ 5606  7653 52617 32936 63413 44748 57308 32031 41368 22782]]
[[ 65891  62448  99286  85537 116993  83867  73551 112135  82476  38327]]
[[    45  85728  95873  44714  53267 102415  16519  37947  20579 115806]]
[[ 86681  76275    106   2259 109710 121787  58516  32951 116808  99931]]
[[ 98496 115496  81846  22665  29747  22796  88267  41599  68974  42280]]
[[20191 66396   643  1305 68855 66841 43306 58030 55253 61

[[   701 112847  71157  89784  66800  85994 121266  71449  68663 109365]]
[[  3069 121439  48780 102822  82343  44020  33039  37736 111229   3491]]
[[ 17228  99268 110004  45631 119423  91172 107145  42004  42321  70126]]
[[   352 113304  63182 110722  37162  79186  32382  59824  65600  86289]]
[[   726  32945 114822 112702  24060 109596  23155  86665  27789  12352]]
[[   132  45581  31155  92874  50598  60637 105791  75500 117548  16627]]
[[  2899  23640  76832   1873  79356  73135 104449 122373  21518  62543]]
[[  2393  30697  49369   7749  14514 100072 103296  15846  21257  43284]]
[[ 93481 109197  21717  25267  63210  21407  87682  79711  56151  45642]]
[[  7000  89809  38098   3178 120668  71786  27009  24891  24535  45279]]
[[ 15776  16444   1698  96739  80502  22005  42397  13060  23464 115971]]
[[ 12607  24149  55581  83055 112424 104714  70814  70445  80634   3574]]
[[ 11303 108359  92451  15146  78083  33815  81791 121830 113006   4268]]
[[  5994  60168  60521  86807  16182  

<span style="color:red">Comparez ces suggestions avec la seconde colonne de play_val (les morceaux effectivement présents). Pour cela vous calculerez le hit@10 qui vaut 1 si le morceau effectivement présent dans la playlist fait partie des 10 propositions (ce score étant moyenné sur l'ensemble de validation) et le NDCG@10 (Normalized Discounted Cumulative Gain) qui prend en compte l'ordre des propositions. Ce second score vaut $1/log2(k+1)$ si la proposition k (k entre 1 et 10) est la proposition correcte et 0 si aucne proposition n'est correcte. Comme précedement vous calulerez le score moyen sur l'ensemble de validation. </span>


In [26]:
from math import * 

def calculate_ndgc_at_10(indexes, real_songs) :
    score = 0
    for i in range(len(indexes)) :
        for j in range(len(indexes[i][0])) :
            if  real_songs[i] == indexes[i][0][j] :
                score += 1 / log(((j + 1) + 1), 2)
    return score / len(indexes)

NDGCatK = calculate_ndgc_at_10(indexes, play_val[:,0])
print(NDGCatK)

1.0


In [27]:
def calculate_hit_at_10(indexes, real_songs) :
    score = 0
    for i in range(len(indexes)) :
        if  real_songs[i] in indexes[i][0] :
            score += 1
    return score / len(indexes)

HitatK = calculate_hit_at_10(indexes, play_val[:,0])
print(HitatK)

1.0


## Tunning des hyper paramètres

<span style="color:red">Vous pouvez maintenant essayer de faire varier les hyper paramètres pour améliorer vos performances. Attention au temps de calcul préparez une grille avec une dizaine de configurations différentes et évaluez chacune d'entre elles sur votre ensemblede validation.
Evaluez les performances finales de la meilleure configuration trouvée sur l'ensemble de test. N'oubliez pas de sauvergader vos résultats.</span>



## Bonus, un peu de musique

Le fichier TrackArtists contient des méta.données sur les morceaux et les artiste pour un sous ensemble des 300000 morceaux présent dans le dataset. Nous pouvons nous en servir pour recherchez le numéro d'un morceau a partir de son titre:

In [ ]:
import pandas as pd
tr_meta=pd.read_csv("./TracksArtists.csv")
joindf = pd.DataFrame({"track_id":tracks_list_ordered[:Vt],"index":range(Vt)})
meta = tr_meta.merge(joindf, left_on="track_id",right_on="track_id")
meta.set_index("index",inplace=True)
meta[["title","name","preview","track_id"]]

In [ ]:
def find_track(title):
    return meta.loc[meta["title"]==title,:].index[0]

tr=find_track("Hexagone")
tr

## Radio

L'api de deeezer permet de récupérer des informations sur les morceaux du dataset a partit de leur id deezer. Parmis ces infos lorsqu'elle est disponnible une url d'écoute d'un extrait gratuit est fournies.

In [ ]:
import urllib.request, json 
def gettrackinfo(number):
    track_url =  "https://api.deezer.com/track/{}".format(tracks_list_ordered[number].split("_")[1])
    with urllib.request.urlopen(track_url) as url:
        data = json.loads(url.read().decode())
    return data
track_apidata = gettrackinfo(find_track("Hexagone"))
track_apidata

Nous pouvons donc nous en servir pour écouter un extrait :

In [ ]:
from IPython.display import display, Audio, clear_output
display(Audio(track_apidata["preview"],autoplay=True))

<span style="color:red">Créez une fonction radio qui prend en entrée un numéro de morceau dans le dataset et lance une serie de nb_track morceaux en tirant aléatoirement dans le voisinage du morceau courant le morceau suivant a écouter. La taille du voisinage sera paramétrable et vous supprimerez des propositions les morceaux déjà écouté. Vous traiterez les exceptions si le morceau ne dispose pas d'extrait disponnible. Vous povez supprimer le morceau courant avec la fonction clear_display.</span>

In [ ]:
import time
def start_radio(seed,nb_candidates,duration,nbsteps=20):
    print(meta.loc[seed,"title"])
    display(Audio(meta.loc[seed,"preview"],autoplay=True))
    time.sleep(duration)
    clear_output()
    already_played = [seed]
    for i in range(nbsteps):
        try:
            # TODO
        except:
            print("track not found")
            pass
        clear_output()

In [ ]:
start_radio(find_track("Hexagone"),5,5,10)